In [ ]:
import numpy as np
import os
from tensorflow import keras

In [ ]:
from tensorflow.keras import models, layers, activations, losses, optimizers
from keras.layers import Flatten, Dense, Reshape, Input , Lambda
from keras.layers import Dense, Conv2D, MaxPooling2D , Flatten
import tensorflow.keras.backend as K
import tensorflow as tf

DIMEN = 128 # dimension of the image

input_shape = ( (DIMEN**2) * 3 , )
convolution_shape = ( DIMEN , DIMEN , 3 )
kernel_size_1 = ( 4 , 4 )
kernel_size_2 = ( 3 , 3 )
pool_size_1 = ( 3 , 3 )
pool_size_2 = ( 2 , 2 )
strides = 1


In [ ]:
data_dimension = 128

X1 = np.load( '')
X2 = np.load( '')
Y = np.load( '') #Read the image 

X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

(36, 49152)
(36, 49152)
(36,)


In [ ]:
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}


In [ ]:
class Recognizer (object) :

	def __init__( self ):

		#tf.logging.set_verbosity( tf.logging.ERROR )

		self.__DIMEN = 128

		input_shape = ( (self.__DIMEN**2) * 3 , )
		convolution_shape = ( self.__DIMEN , self.__DIMEN , 3 )
		kernel_size_1 = ( 4 , 4 )
		kernel_size_2 = ( 3 , 3 )
		pool_size_1 = ( 3 , 3 )
		pool_size_2 = ( 2 , 2 )
		strides = 1

		seq_conv_model = [

			Reshape( input_shape=input_shape , target_shape=convolution_shape),

			Conv2D( 32, kernel_size=kernel_size_1 , strides=strides , activation=activations.relu ),
			Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=activations.relu),
			MaxPooling2D(pool_size=pool_size_1, strides=strides ),
      Conv2D( 32, kernel_size=kernel_size_1 , strides=strides , activation=activations.relu ),
			Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=activations.relu),
			MaxPooling2D(pool_size=pool_size_1, strides=strides ),

			Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=activations.relu ),
			Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=activations.relu ),
			MaxPooling2D(pool_size=pool_size_2 , strides=strides),

			Flatten(),

			Dense( 64 , activation=activations.sigmoid )

		]

		seq_model = tf.keras.Sequential( seq_conv_model )

		input_x1 = Input( shape=input_shape )
		input_x2 = Input( shape=input_shape )

		output_x1 = seq_model( input_x1 )
		output_x2 = seq_model( input_x2 )

		distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
		outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
		self.__model = models.Model( [ input_x1 , input_x2 ] , outputs )

		self.__model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001),metrics=['accuracy'])



	def fit(self, X, Y ,  hyperparameters  ):
		initial_time = time.time()
		h = self.__model.fit( X  , Y ,
						 batch_size=hyperparameters[ 'batch_size' ] ,
						 epochs=hyperparameters[ 'epochs' ] ,
						 callbacks=hyperparameters[ 'callbacks'],
						 validation_data=hyperparameters[ 'val_data' ]
						 )
		final_time = time.time()
		eta = ( final_time - initial_time )
		time_unit = 'seconds'
		if eta >= 60 :
			eta = eta / 60
			time_unit = 'minutes'
		self.__model.summary( )
		print( 'Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit ) )
    #return h

	

	def evaluate(self , test_X , test_Y  ) :
		return self.__model.evaluate(test_X, test_Y)


	def predict(self, X  ):
		predictions = self.__model.predict( X  )
		return predictions


	def summary(self):
		self.__model.summary()
    #return 0 #self.__model
  
  

	def save_model(self , file_path ):
		self.__model.save(file_path )


	def load_model(self , file_path ):
		self.__model = models.load_model(file_path)
  


In [ ]:
import time
recognizer = Recognizer()
#recognizer.load_model('models/model.h5')

parameters = {
    'batch_size' : 6 ,
    'epochs' : 3 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)
#recognizer.save_model('models/model.h5')

Epoch 1/3
6/6 [==============================] - 22s 3s/step - loss: 0.6419 - accuracy: 0.5714
Epoch 2/3
6/6 [==============================] - 20s 3s/step - loss: 0.3549 - accuracy: 0.8992
Epoch 3/3
6/6 [==============================] - 21s 3s/step - loss: 0.2623 - accuracy: 1.0000
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 49152)]      0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 49152)]      0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 64)           47001408    input_11[0][0]                   
     